# Loading required libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')
import random
import numpy as np
import torch
def set_seed(seed=1):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

set_seed()

# Download dataset from Github

In [2]:
!wget https://raw.githubusercontent.com/giordamaug/BIONETdatasets/main/CSV/integratedcrispr/edges_integrated2.csv
!wget https://raw.githubusercontent.com/giordamaug/BIONETdatasets/main/CSV/integratedcrispr/nodes_integrated2_update_attr_label_avana0_wang_crispr.csv

--2022-04-30 07:27:38--  https://raw.githubusercontent.com/giordamaug/BIONETdatasets/main/CSV/integratedcrispr/edges_integrated2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39025931 (37M) [text/plain]
Saving to: ‘edges_integrated2.csv’

edges_integrated2.c 100%[===================>]  37.22M  --.-KB/s    in 0.1s    

2022-04-30 07:27:39 (269 MB/s) - ‘edges_integrated2.csv’ saved [39025931/39025931]

--2022-04-30 07:27:39--  https://raw.githubusercontent.com/giordamaug/BIONETdatasets/main/CSV/integratedcrispr/nodes_integrated2_update_attr_label_avana0_wang_crispr.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com

# Load the attributes and labels from CSV

All node/gene attributes and labels are in one file. Firs we devide attributes (numeri) from labels (non-numeric) columns.



In [3]:
import pandas as pd
from sklearn import preprocessing
node_name, label_name = 'name', 'label_CS_ACH_most_freq'
# read node attributes
df = pd.read_csv('/content/nodes_integrated2_update_attr_label_avana0_wang_crispr.csv', 
                 sep='\t', index_col=node_name)   # load csv and set 'name' as index
x = df.select_dtypes(include=np.number)           # Attributes are numeric columns
labels = df.select_dtypes(exclude=np.number)      # labels are not numeric columns

# Set the label
The label is one column or a grouping of values from one label column. In the paper we considered the ten values of `label_CS_ACH_most_freq`, and we grouped as following:
*   `CS0` = `E`ssential class
*   `CS6`-`CS9` = `NE` non essential class

The new label name is `CS0_vs_CS6-9`





In [4]:
E_class, NE_class = ['CS0'], ['CS6', 'CS7', 'CS8', 'CS9']
new_label_name = 'CS0_vs_CS6-9'
labels[new_label_name] = labels.apply(lambda row: 'E' if row[label_name] in E_class \
                                      else 'NE' if row[label_name] in NE_class \
                                      else row[label_name], axis=1)
labels = labels[labels[new_label_name].isin(['E', 'NE']) == True]       # drop any row contaning NaN or SC1-SC5 as value
genes = labels.index.values
print(f'Selected {len(genes)} genes')

Selected 3814 genes


# Encode the label

In [5]:
from sklearn import preprocessing
from collections import Counter
encoder = preprocessing.LabelEncoder()
y = encoder.fit_transform(labels[new_label_name].values)  
classes_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
print(classes_mapping, Counter(y))

{'E': 0, 'NE': 1} Counter({1: 3069, 0: 745})


# Load the PPI network
The PPI networks is loaded from a CSV file, where
*   `A` is the column name for edge source (gene name)
*   `B` is the column name for edge target (gene name)
*   `weight` is the column name for edge weight
Only some method use the PPI netoworks, as an example all GCN methods, and Node2Vec.



In [6]:
ppi = pd.read_csv('/content/edges_integrated2.csv', sep='\t')              # read ppi from CSV file
ppi = ppi.loc[((ppi['A'].isin(genes)) & (ppi['B'].isin(genes)))]           # reduce PPI only to selected nodes/genes
idxlbl = labels.reset_index(drop=True)
idxlbl[node_name] = labels.index
map_gene_to_idx = { v[node_name]: i  for i,v in idxlbl.to_dict('Index').items() }
vfunc = np.vectorize(lambda t: map_gene_to_idx[t])
edges_index = torch.from_numpy(vfunc(ppi[['A','B']].to_numpy().T)) 

# Select attributes to be used
We identified three sets of attributes:
1. bio attributes, related to gene information (such as, expression, etc.)
2. net attributes, derived from role of gene/node in the network (such as, degree, centrality, etc.)
3. GTEX-* attribute, additional biological information of genes 

In this code snippets the sets of attibutes are defines, and you may choose to subtract some of them from the matrix of node attributes 


In [7]:
#@title Choose attributes { form-width: "20%" }
import re
bio_attr = True #@param {type:"boolean"}
net_attr = True #@param {type:"boolean"}
gtex_attr = True #@param {type:"boolean"}
bio_attributes = ['gc_content', 'Gtex_kidney', 'gene_disease_ass_count', 'oncodb_expression','orth_count', 'gene_length', \
       'HPA_kidney', 'mf_coal', 'bp_coal', 'cc_coal', 'biogrid_coal', 'kegg_coal', 'reactome_coal', 'ucsc_tfbs_coal', \
       'up_tissue_coal', 'transcript_count']  if bio_attr else []
net_attributes = ['degree', 'ecc', 'clos', 'betw', 'eigen', 'hub', 'trans', 'PR', 'triangles_numb', 'motif1', \
        'motif2', 'motif3', 'motif5', 'strength'] if net_attr else []
r = re.compile('^GTEX*')
gtex_attributes = list(filter(r.match, x.columns)) if gtex_attr else []
x = x.filter(items=bio_attributes+gtex_attributes+net_attributes)

## Normalize attributes
In this snippet of code the matrix of node attributes is corrected by filling NaN with themean in the columns, while Infinte value with maximum. 

The attribute matrix is also reduce by removing all rows correspondning to node not considered, i.e. deleted becaus no label was associated to them.

In [8]:
#@title Normalization modes { form-width: "30%" }
normalize_node = "zscore" #@param ["", "zscore", "minmax"]
print(f'Fixing NaN and infinity in X matrix...', end='')
print(f'Found {x.isnull().sum().sum()} NaN values and {np.isinf(x).values.sum()} Inf values')
highest_non_inf = x.max().loc[lambda v: v<np.Inf].max()    # fix infinity (replace with max)
x.replace(np.Inf, highest_non_inf)
for col in x.columns[x.isna().any()].tolist():
  mean_value=x[col].mean()          # Replace NaNs in column with the  mean of values in the same column
  if mean_value is not np.nan:
    x[col].fillna(value=mean_value, inplace=True)
  else:
    x = x.drop(col, 1)
if normalize_node == 'minmax':
  print("X attributes normalization (minmax)...")
  x = (x-x.min())/(x.max()-x.min())
elif normalize_node == 'zscore':
  print("X attributes normalization (zscore)...")
  x = (x-x.mean())/x.std()
x = x.loc[genes]
print(f'New attribute matrix x{x.shape}')

Fixing NaN and infinity in X matrix...Found 15919 NaN values and 0 Inf values
X attributes normalization (zscore)...
New attribute matrix x(3814, 119)


# k-fold cross validation with: SVM, RF, XGB, MLP, RUS

In [17]:
#@title Choose classifier { run: "auto", form-width: "20%" }
method = "SVM" #@param ["SVM", "XGB", "RF", "MLP", "RUS", "DUMMY"]
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_selector as selector
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.neural_network import MLPClassifier
import sys
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split
from tqdm import tqdm
from sklearn.metrics import *
from sklearn.ensemble import AdaBoostClassifier
from imblearn.ensemble import RUSBoostClassifier
from sklearn.dummy import DummyClassifier
seed = 123
nfolds = 5
kf = KFold(n_splits=nfolds)
accuracies, mccs = [], []
cm = [[0,0],[0,0]]
X = x.to_numpy()

if method == 'SVM':
  clf = SVC(random_state=seed, probability=True)
elif method == 'MLP':
  clf = MLPClassifier(random_state=seed)
elif method == 'RF':
  clf = RandomForestClassifier(random_state=seed)
elif method == 'XGB':
  clf = XGBClassifier(random_state=seed)
elif method == 'RUS':
  clf = RUSBoostClassifier(random_state=seed)
elif method == 'DUMMY':
  clf = DummyClassifier(random_state=seed)

cma = np.array([[0,0],[0,0]])
mm = np.array([], dtype=np.int)
predictions = np.array([])
columns_names = ["Accuracy","BA", "Sensitivity", "Specificity","MCC", 'CM']
scores = pd.DataFrame(columns=columns_names)
for fold, (train_idx, test_idx) in enumerate(tqdm(kf.split(np.arange(len(X))), total=kf.get_n_splits(), desc=f"{nfolds}-fold")):
    train_x, train_y, test_x, test_y = X[train_idx], y[train_idx], X[test_idx], y[test_idx],
    mm = np.concatenate((mm, test_idx))
    probs = clf.fit(train_x, train_y).predict(test_x)
    #preds = np.argmax(probs, axis=1)
    preds = (probs > 0.5) * 1
    cm = confusion_matrix(test_y, preds)
    cma += cm
    predictions = np.concatenate((predictions, preds))
    #p = np.concatenate([genes[test_idx].reshape(-1,1), probs], axis=1)
    scores = scores.append(pd.DataFrame([[accuracy_score(test_y, preds), balanced_accuracy_score(test_y, preds), 
        cm[0,0]/(cm[0,0]+cm[0,1]), cm[1,1]/(cm[1,0]+cm[1,1]), 
        matthews_corrcoef(test_y, preds), cm]], columns=columns_names, index=[fold]))
df_scores = pd.DataFrame(scores.mean(axis=0)).T
df_scores.index=[f'{method}']
df_scores['CM'] = [cma]
print(df_scores.to_latex())
df_scores
p = np.zeros(len(y))
p[mm] = predictions

5-fold: 100%|██████████| 5/5 [00:15<00:00,  3.08s/it]

\begin{tabular}{lrrrrrl}
\toprule
{} &  Accuracy &        BA &  Sensitivity &  Specificity &       MCC &                        CM \\
\midrule
SVM &  0.864973 &  0.697553 &     0.424777 &     0.970328 &  0.510203 &  [[320, 425], [90, 2979]] \\
\bottomrule
\end{tabular}



# Print Predictions

In [18]:
print(np.unique(y+p, return_counts=True))
labels['predictions'] = p
labels[(labels['predictions'] == 0 ) & ( labels['CS0_vs_CS6-9'] == 'E')].index
f= open(f'{method}_Egenes.csv', "w")
f.write('\n'.join([str(e) for e in list(labels[(labels['predictions'] == 0 ) & ( labels['CS0_vs_CS6-9'] == 'E')].index)]))
f.close()
labels[['CS0_vs_CS6-9', 'predictions']].to_csv(f'{method}_Predictions.csv')

(array([0., 1., 2.]), array([ 320,  515, 2979]))
